In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import string
import re
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score

In [2]:
#read data for local machine
curr_dir= os.getcwd()
train_full = pd.read_csv(curr_dir+"//Dataset/train_with_brand_from_name.csv",encoding = 'ISO-8859-1')
#test = pd.read_csv(curr_dir+"//Dataset/test_with_brand_from_name.csv",encoding = 'ISO-8859-1')

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train, test = train_test_split(train_full, test_size=0.2)

In [4]:
train.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3
725111,1265410,Stride Rite Mary Jane & Seahorse flips,2,Stride Rite,14.0,0,Stride Rite Mary Jane Silver Size 3T. Outgrew ...,Kids,Girls 0-24 Mos,Shoes
921784,493859,Pink Victoria's Secret workout tops,3,Secret,29.0,0,2 for the price of one! Like new / gently used...,Women,Tops & Blouses,"Tank, Cami"
1006176,17245,1 Pink Sport Short,1,NaN,12.0,1,No description yet,Women,Athletic Apparel,Shorts
1333777,1025224,Jewelry box,2,NaN,10.0,1,"Brand new jewelry box, I don't have much jewel...",Women,Jewelry,Earrings
671246,1171159,Silky Sunflower Dress,3,Vintage,12.0,0,Silky dark blue Summer Dress with Sunflower pr...,Women,Dresses,Knee-Length


In [5]:
train.shape

(1186028, 10)

In [6]:
test.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3
756945,1320539,Bath and Body WallFlower Sea Turtle,2,Bath & Body Works,8.0,0,Bath and Body Brown Sea Turtle Wall Flower.,Home,Home Décor,Home Fragrance
771885,1346622,Beatles yellow submarine hot wheel,1,Hot Wheels,10.0,0,The Beatles yellow submarine hot wheel. Brand ...,Vintage & Collectibles,Toy,Car
1333173,1023305,10/12 NWT Girls Pretty Car PJ Sleep Set,1,NaN,7.0,1,Super cute NWT 10/12 Girls size Large Pajama S...,Kids,Girls (4+),Other
1017872,52641,Bronzer Shimmer,1,NaN,6.0,1,??These irridescent powders extend the life of...,Beauty,Makeup,Makeup Palettes
96619,168984,Scentsy Farmstand Pumpkin,1,Scentsy,20.0,1,3 Scentsy Bars (Farmstand Pumpkin) 1 Room Spra...,Home,Home Décor,Home Fragrance


In [7]:
test.shape

(296507, 10)

In [8]:
train = train.rename(columns = {'train_id':'id'})
test = test.rename(columns = {'train_id':'id'})

In [9]:
train['is_train'] = 1
test['is_train'] = 0

In [10]:
train1 = train.drop(['price'],axis =1)
test1 = test.drop(['price'],axis=1)

In [11]:
train_test_combine = pd.concat([train1,test1],axis = 0)

In [12]:
train_test_combine.head()

,id,name,item_condition_id,brand_name,shipping,item_description,cat_1,cat_2,cat_3,is_train
725111,1265410,Stride Rite Mary Jane & Seahorse flips,2,Stride Rite,0,Stride Rite Mary Jane Silver Size 3T. Outgrew ...,Kids,Girls 0-24 Mos,Shoes,1
921784,493859,Pink Victoria's Secret workout tops,3,Secret,0,2 for the price of one! Like new / gently used...,Women,Tops & Blouses,"Tank, Cami",1
1006176,17245,1 Pink Sport Short,1,NaN,1,No description yet,Women,Athletic Apparel,Shorts,1
1333777,1025224,Jewelry box,2,NaN,1,"Brand new jewelry box, I don't have much jewel...",Women,Jewelry,Earrings,1
671246,1171159,Silky Sunflower Dress,3,Vintage,0,Silky dark blue Summer Dress with Sunflower pr...,Women,Dresses,Knee-Length,1


In [13]:
train_test_combine.shape

(1482535, 10)

In [14]:
train_test_combine['cat_3'].describe()

count                     1471819
unique                        864
top       Pants, Tights, Leggings
freq                        60177
Name: cat_3, dtype: object

In [15]:
train_test_combine.cat_3.fillna('No',inplace = True)
train_test_combine.cat_1.fillna('No',inplace = True)
train_test_combine.cat_2.fillna('No',inplace = True)
train_test_combine.brand_name.fillna('No',inplace = True)

In [16]:
cat_1_list = list(train_test_combine['cat_1'].unique())

In [111]:
cat_1_list

['Kids',
 'Women',
 'Men',
 'Electronics',
 'Handmade',
 'Beauty',
 'Home',
 'Sports & Outdoors',
 'No',
 'Vintage & Collectibles',
 'Other']

In [18]:
df_1 = train_test_combine[train_test_combine['cat_1'] == 'Beauty']
df_2 = train_test_combine[train_test_combine['cat_1'] == 'Electronics']
df_3 = train_test_combine[train_test_combine['cat_1'] == 'Handmade']
df_4 = train_test_combine[train_test_combine['cat_1'] == 'Home']
df_5 = train_test_combine[train_test_combine['cat_1'] == 'Kids']
df_6 = train_test_combine[train_test_combine['cat_1'] == 'Other']
df_7 = train_test_combine[train_test_combine['cat_1'] == 'Sports & Outdoors']
df_8 = train_test_combine[train_test_combine['cat_1'] == 'Vintage & Collectibles']
df_9 = train_test_combine[train_test_combine['cat_1'] == 'Women']
df_10 = train_test_combine[train_test_combine['cat_1']== 'No Brand']

In [19]:
df_1.head()

,id,name,item_condition_id,brand_name,shipping,item_description,cat_1,cat_2,cat_3,is_train
296315,516767,Urban Decay Naked 2,1,Urban Decay,0,"Urban Decay Naked 2 palette- in box, untouched...",Beauty,Makeup,Makeup Palettes,1
1234819,721768,Krasr black head Remover,2,NaN,0,New open box Black head Remover,Beauty,Skin Care,Face,1
950332,452859,Clarisonic Mia,2,Clarisonic,0,"Works perfect, will come with charger. This Is...",Beauty,Skin Care,Face,1
653251,1139695,Nars soft matte Concealer,3,Nars,0,barely used: color custard,Beauty,Makeup,Face,1
539658,941343,Urban Decay x Gwen Stefani Pallet,3,Urban Decay,1,100% Authentic Lightly used 1-2 times no packa...,Beauty,Makeup,Makeup Palettes,1


In [20]:
def work_on_each_dataframe(df_w):
    #one hot encoding on one dataset
    df_w.drop(['cat_1'],axis=1,inplace= True)
    df_w = pd.get_dummies(df_w, columns=["cat_2"])
    df_w = pd.get_dummies(df_w, columns=["cat_3"])
    
    df_w.name = df_w.name.astype('category')
    #df_w.brand_name = df_w.brand_name.astype('category')
    
    #label Encoding
    #df_w.brand_name = df_w.brand_name.cat.codes
    df_w.name = df_w.name.cat.codes

    df_w = df_w.drop(['item_description'],axis = 1)
    
    return df_w

In [21]:
df_1 = work_on_each_dataframe(df_1)
df_2 = work_on_each_dataframe(df_2)
df_3 = work_on_each_dataframe(df_3)
df_4 = work_on_each_dataframe(df_4)
df_5 = work_on_each_dataframe(df_5)
df_6 = work_on_each_dataframe(df_6)
df_7 = work_on_each_dataframe(df_7)
df_8 = work_on_each_dataframe(df_8)
df_9 = work_on_each_dataframe(df_9)
df_10 = work_on_each_dataframe(df_10)

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
df_1.shape

(207828, 59)

In [23]:
df_1.head()

,id,name,item_condition_id,brand_name,shipping,is_train,cat_2_Bath & Body,cat_2_Fragrance,cat_2_Hair Care,cat_2_Makeup,...,cat_3_Shampoo & Conditioner Sets,cat_3_Shampoo Plus Conditioner,cat_3_Shampoos,cat_3_Styling Products,cat_3_Styling Tools,cat_3_Sun,cat_3_Toiletry Kits,cat_3_Tweezers,cat_3_Waxing,cat_3_Women
296315,516767,154164,1,Urban Decay,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1234819,721768,80882,2,NaN,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
950332,452859,43748,2,Clarisonic,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653251,1139695,114496,3,Nars,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
539658,941343,154917,3,Urban Decay,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [106]:
def model_train(df_m):
    df_test= df_m.loc[df_m['is_train']==0]
    df_train= df_m.loc[df_m['is_train']==1]
    
    df_test = df_test.drop(['is_train'],axis=1)
    df_train = df_train.drop(['is_train'],axis=1)
    
    df_train['price'] = train.price
    df_test['price'] = test.price
    #df_train['price'] = df_train['price'].apply(lambda x: np.log(x) if x>0 else x)
    return df_train,df_test

In [107]:
df_1_train, df_1_test = model_train(df_1)
df_2_train, df_2_test = model_train(df_2)
df_3_train, df_3_test = model_train(df_3)
df_4_train, df_4_test = model_train(df_4)
df_5_train, df_5_test = model_train(df_5)
df_6_train, df_6_test = model_train(df_6)
df_7_train, df_7_test = model_train(df_7)
df_8_train, df_8_test = model_train(df_8)
df_9_train, df_9_test = model_train(df_9)
df_10_train, df_10_test = model_train(df_10)

In [110]:
df_2_train.head()

,id,name,item_condition_id,brand_name,shipping,cat_2_Cameras & Photography,"cat_2_Car Audio, Video & GPS",cat_2_Cell Phones & Accessories,cat_2_Computers & Tablets,cat_2_Media,...,cat_3_Portable Audio & Accessories,"cat_3_Printers, Scanners & Supplies",cat_3_Replacement Parts & Tools,cat_3_Screen Protectors,cat_3_Strategy Guides,cat_3_Televisions,cat_3_Tripods & Supports,cat_3_VHS,cat_3_Video Gaming Merchandise,price
308304,537505,34828,3,HP,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,9.0
111396,194482,12528,1,Apple,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,16.0
1120663,370144,16336,1,NaN,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,14.0
374157,652184,33710,3,Xbox,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30.0
523468,913057,52902,4,Bose,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19.0


In [26]:
df_1_test.head()

,id,name,item_condition_id,brand_name,shipping,cat_2_Bath & Body,cat_2_Fragrance,cat_2_Hair Care,cat_2_Makeup,cat_2_Other,...,cat_3_Shampoo Plus Conditioner,cat_3_Shampoos,cat_3_Styling Products,cat_3_Styling Tools,cat_3_Sun,cat_3_Toiletry Kits,cat_3_Tweezers,cat_3_Waxing,cat_3_Women,price
1017872,52641,36245,1,NaN,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,6.0
185088,323016,48924,1,Bath & Body Works,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,55.0
731712,1276665,44850,1,Clinique,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,15.0
1023986,71546,103987,1,NaN,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,10.0
1224290,689356,99857,1,NaN,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,8.0


In [27]:
df_1_train.head()

,id,name,item_condition_id,brand_name,shipping,cat_2_Bath & Body,cat_2_Fragrance,cat_2_Hair Care,cat_2_Makeup,cat_2_Other,...,cat_3_Shampoo Plus Conditioner,cat_3_Shampoos,cat_3_Styling Products,cat_3_Styling Tools,cat_3_Sun,cat_3_Toiletry Kits,cat_3_Tweezers,cat_3_Waxing,cat_3_Women,price
296315,516767,154164,1,Urban Decay,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,25.0
1234819,721768,80882,2,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,31.0
950332,452859,43748,2,Clarisonic,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,29.0
653251,1139695,114496,3,Nars,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,14.0
539658,941343,154917,3,Urban Decay,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,25.0


In [28]:
'''
brand_group: Dataframe comprising mean of each brand
brand_group1: Training Dataset, in which we will append the brand_weight
brand_group11: Testing Dataset, in which we will append the brand_weight
brand_group2: a temperory variable for train data
brand_group22: a temperory variable for test data 
'''
def brand_weight_train(df_train1,df_test1):
    brand_group = df_train1.groupby(['brand_name'])['price'].mean().to_frame()
    brand_group['brand_name'] =brand_group.index
    i=0
    j=0
    for index1,row1 in brand_group.iterrows():
        if i ==0:
            try:
                brand_group1 = df_train1[df_train1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
                brand_group1.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
                i=1
            except:
                pass
        else:
            try:
                brand_group2 = df_train1[df_train1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
                brand_group2.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
                brand_group1 = brand_group1.append(brand_group2)
            except:
                pass
        
        if j==0:
            try:
                brand_group11 = df_test1[df_test1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
                brand_group11.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
                j=1
            except:
                pass
            
        else:
            try:
                brand_group22 = df_test1[df_test1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
                brand_group22.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
                brand_group11 = brand_group11.append(brand_group22)
            except:
                pass
    
    return brand_group1, brand_group11

In [74]:
%%time
#df_1_train,df_1_test = brand_weight_train(df_1_train,df_1_test)
df_2_train,df_2_test = brand_weight_train(df_2_train,df_2_test)
#df_3_train,df_3_test = brand_weight_train(df_3_train,df_3_test)
#df_4_train,df_4_test = brand_weight_train(df_4_train,df_4_test)
#df_5_train,df_5_test = brand_weight_train(df_5_train,df_5_test)
#df_6_train,df_6_test = brand_weight_train(df_6_train,df_6_test)
#df_7_train,df_7_test = brand_weight_train(df_7_train,df_7_test)
#df_8_train,df_8_test = brand_weight_train(df_8_train,df_8_test)
#df_9_train,df_9_test = brand_weight_train(df_9_train,df_9_test)
#df_10_train,df_10_test = brand_weight_train(df_10_train,df_10_test)

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Wall time: 5min 56s


In [30]:
def encode_brand_name_price(df__train,df__test):
    df__train.brand_name = df__train.brand_name.astype('category')
    df__test.brand_name = df__test.brand_name.astype('category')
    df__train.brand_name = df__train.brand_name.cat.codes
    df__test.brand_name = df__test.brand_name.cat.codes
    df__train['price'] = df__train['price'].apply(lambda x: np.log(x) if x>0 else x)
    df__test['price'] = df__test['price'].apply(lambda x: np.log(x) if x>0 else x)
    return df__train,df__test

In [31]:
df_1_train,df_1_test = encode_brand_name_price(df_1_train,df_1_test)
#df_2_train,df_2_test = encode_brand_name_price(df_2_train,df_2_test)
#df_3_train,df_3_test = encode_brand_name_price(df_3_train,df_3_test)
#df_4_train,df_4_test = encode_brand_name_price(df_4_train,df_4_test)
#df_5_train,df_5_test = encode_brand_name_price(df_5_train,df_5_test)
#df_6_train,df_6_test = encode_brand_name_price(df_6_train,df_6_test)
#df_7_train,df_7_test = encode_brand_name_price(df_7_train,df_7_test)
#df_8_train,df_8_test = encode_brand_name_price(df_8_train,df_8_test)
#df_9_train,df_9_test = encode_brand_name_price(df_9_train,df_9_test)
#df_10_train,df_10_test = encode_brand_name_price(df_10_train,df_10_test)

In [48]:
def Random_forest(df_train,df_test):
    x_train,y_train = df_train.drop(['price'],axis =1),df_train.price
    x_test,y_test = df_test.drop(['price'],axis =1),df_test.price
    y_train = pd.Series(np.exp(y_train))
    y_test =pd.Series(np.exp(y_test))
    m = RandomForestRegressor(n_jobs=-1,min_samples_leaf=3,n_estimators=200)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    #preds = pd.Series(np.exp(preds))
    #print(m.score(preds,y_test))
    print(r2_score(y_test, preds))

In [49]:
Random_forest(df_1_train,df_1_test)
#Random_forest(df_2_train,df_2_test)
#Random_forest(df_3_train,df_3_test)
#Random_forest(df_4_train,df_4_test)
#Random_forest(df_5_train,df_5_test)
#Random_forest(df_6_train,df_6_test)
#Random_forest(df_7_train,df_7_test)
#Random_forest(df_8_train,df_8_test)
#Random_forest(df_9_train,df_9_test)
#Random_forest(df_10_train,df_10_test)

0.100358770887


In [60]:
brand_group = df_2_train.groupby(['brand_name'])['price'].mean().to_frame()

In [64]:
brand_group.columns = ['brand_weight']

In [67]:
brand_group['brand_name'] = brand_group.index

In [70]:
brand_group.head()

,brand_weight,brand_name
brand_name,,
1byone,51.00,1byone
2K Games,9.00,2K Games
3M®,7.00,3M®
47,22.50,47
A Bathing Ape,9.75,A Bathing Ape


In [73]:
df_2_train[df_2_train['id']==1118841]

,id,name,item_condition_id,brand_name,shipping,cat_2_Cameras & Photography,"cat_2_Car Audio, Video & GPS",cat_2_Cell Phones & Accessories,cat_2_Computers & Tablets,cat_2_Media,...,cat_3_Portable Audio & Accessories,"cat_3_Printers, Scanners & Supplies",cat_3_Replacement Parts & Tools,cat_3_Screen Protectors,cat_3_Strategy Guides,cat_3_Televisions,cat_3_Tripods & Supports,cat_3_VHS,cat_3_Video Gaming Merchandise,price
641330,1118841,34824,3,HP,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,14.0


In [100]:
df_2_train.head()

,id,name,item_condition_id,brand_name,shipping,cat_2_Cameras & Photography,"cat_2_Car Audio, Video & GPS",cat_2_Cell Phones & Accessories,cat_2_Computers & Tablets,cat_2_Media,...,"cat_3_Printers, Scanners & Supplies",cat_3_Replacement Parts & Tools,cat_3_Screen Protectors,cat_3_Strategy Guides,cat_3_Televisions,cat_3_Tripods & Supports,cat_3_VHS,cat_3_Video Gaming Merchandise,price,brand_weight
621979,1084814,1788,1,1byone,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,51.0,51.0
152408,266199,59545,1,2K Games,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9.0,9.0
281681,491232,10339,1,3M®,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,7.0,7.0
988746,852702,99049,3,47,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,29.0,22.5
988757,1224886,6265,3,47,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,16.0,22.5


In [68]:
ab= pd.merge(df_2_train,brand_group, on='brand_name')

In [99]:
ab.head()

,id,name,item_condition_id,brand_name,shipping,cat_2_Cameras & Photography,"cat_2_Car Audio, Video & GPS",cat_2_Cell Phones & Accessories,cat_2_Computers & Tablets,cat_2_Media,...,"cat_3_Printers, Scanners & Supplies",cat_3_Replacement Parts & Tools,cat_3_Screen Protectors,cat_3_Strategy Guides,cat_3_Televisions,cat_3_Tripods & Supports,cat_3_VHS,cat_3_Video Gaming Merchandise,price,brand_weight
0,537505,34828,3,HP,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,9.0,84.509972
1,1118841,34824,3,HP,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,14.0,84.509972
2,114018,36391,3,HP,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,105.0,84.509972
3,1149784,34745,3,HP,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,79.0,84.509972
4,478439,34837,2,HP,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,15.0,84.509972


In [95]:
abc = df_2_train[~df_2_train.index.isin(ab.index)]

In [96]:
abc.shape

(59240, 65)

In [97]:
62621-59451

3170

In [104]:
abc = df_2_train[df_2_train.isin(ab)]

In [105]:
abc.shape

(0, 65)